In [59]:
from Manager import *
from VisionUtils import *
tf.enable_eager_execution()

In [60]:
IMG_RT_PATH = 'data/whole_images/'
FEATURE_RT_PATH = "data/processed/other_image_features/"
FEATURE_AUG_RT_PATH = "data/image_aug_features/"
IMG_AUG_RT_PATH = 'data/images_aug/'

#img_list = pd.read_csv(IMG_RT_PATH + "test_list.TXT")
feature_extractor = load_model("Models\FaceNet/Facenet_keras.h5")
face_detector = MTCNN()

W0925 12:03:41.859815 23432 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


In [61]:
import os 
img_path_list = []
folder_list = os.listdir(IMG_RT_PATH)
for FOLDER in folder_list:
    if '.' in FOLDER:
        continue
    image_list_names = os.listdir(IMG_RT_PATH + FOLDER)
    for NAME in image_list_names:
        img_path_list.append(FOLDER+'/'+NAME)

In [62]:
img_list = pd.DataFrame({"path": img_path_list})

In [63]:
iman = ImageManager(img_list, feature_extractor = feature_extractor,
                    face_detector = face_detector,
                    IMG_RTPATH= IMG_RT_PATH)

In [64]:
iman.split(train_size = 0.75)

In [65]:
img_path_list[0]

'AbdA/AbdA_00001_m_31_i_fr_nc_no_2016_2_e0_Gn_h.jpg'

In [66]:
input_shape = feature_extractor.input_shape[1:]

In [67]:
def extract_feature( feature_extractor, img, bb):
    """
    Extract features given img, bounding box and feature extractor
    """
    insz = feature_extractor.input_shape
    img = crop_face(img, bb)
    img = cv2.resize(img, (160,160), interpolation = cv2.INTER_AREA)
    img = normalize(img).reshape((1, insz[1],insz[2], insz[3]))
    image_feature = feature_extractor.predict(img)
    return image_feature

In [72]:
img_path_list[0].split('/')[0]

'AbdA'

In [83]:
import tqdm
img_path_list[0][:-4]

'AbdA/AbdA_00001_m_31_i_fr_nc_no_2016_2_e0_Gn_h'

In [84]:
for IMG_PATH in tqdm.tqdm(img_path_list):
    if (os.path.isfile(IMG_RT_PATH + IMG_PATH)):
        if '._' in IMG_PATH:
            continue
        img = cv2.imread(IMG_RT_PATH + IMG_PATH)
    else: 
        continue
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    bb = detect_faces(img, face_detector)
    if(len(bb) != 1):
        continue
    feature = extract_feature(feature_extractor, img, bb[0])
    if(os.path.isdir(FEATURE_RT_PATH + IMG_PATH.split('/')[0])):
        np.save( FEATURE_RT_PATH + IMG_PATH[:-4]+ ".npy", feature)
    else:
        os.mkdir(FEATURE_RT_PATH + IMG_PATH.split('/')[0])
        np.save( FEATURE_RT_PATH + IMG_PATH[:-4] + ".npy", feature)
    
        
        

100%|██████████████████████████████████████████████████████████████████████████| 83152/83152 [1:55:42<00:00,  4.56it/s]


In [79]:
img = cv2.imread(IMG_RT_PATH + IMG_PATH)

In [82]:
IMG_RT_PATH 

'data/whole_images/'